In [1]:
import sys
import os 

sys.path.insert(0, "/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch")
sys.path.insert(0, "/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch/cpc/eval")
#sys.path.insert(0, "/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_audio/cpc/eval")
#sys.path.insert(0, "/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_audio/cpc")

sys.path.insert(0, "/gpfsdswork/projects/rech/ank/ucv88ce/repos/zerospeech2021_baseline/scripts")

print(sys.path)

['/gpfsdswork/projects/rech/ank/ucv88ce/repos/zerospeech2021_baseline/scripts', '/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch/cpc/eval', '/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch', '/gpfsdswork/projects/rech/ank/ucv88ce/projects/MultilingualCPC/analysis/clusters_units', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python38.zip', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8/lib-dynload', '', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8/site-packages', '/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8/site-packages/IPython/extensions', '/gpfs7kw/linkhome/rech/genini01/ucv88ce/.ipython']


In [2]:
import glob
import pickle
from tqdm import tqdm
import os

In [3]:
import pyximport
pyximport.install()

import json
from os.path import exists, splitext, basename
from pathlib import Path

import torch
from cpc.criterion.clustering.clustering import loadClusterModule
import cpc.eval.eval_ABX

from cpc.eval.eval_ABX_clustering import ClusteringFeatures

from cpc.feature_loader import buildFeature, FeatureModule, loadModel


/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [11]:
#find a way to use jupyter notebook with gpu
#srun --ntasks=1 --cpus-per-task=10 --gres=gpu:1 --hint=nomultithread --time=01:00:00 --pty bash 

In [4]:
wav_fr = [x for x in glob.glob("/gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/fr/*.wav")]
wav_en = [x for x in glob.glob("/gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/en/*.wav")]

In [4]:
audio='/gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/fr/common_voice_fr_17314976.wav'
kmeans_cp="/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/50h/00/kmeans50/checkpoint_last.pt"

In [10]:
os.makedirs("/gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/cpc_small/features/CV_en", exist_ok=True) 

%run /gpfsdswork/projects/rech/ank/ucv88ce/repos/zerospeech2021_baseline/scripts/build_CPC_features.py \
    "/gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/cpc_small/checkpoint_25.pt" \
    "/gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/en" \
    "/gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/features/cpc_small/CV_en" \
    --file_extension wav --gru_level 2

Building CPC features from /gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/en

Looking for all wav files in /gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/en
Loaded from cache /gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/en/_seqs_cache.txt successfully
Done! Found 7042 files!

Creating the output directory at /gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/features/cpc_small/CV_en
Writing args to /gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/features/cpc_small/CV_en/_info_args.json

Loading CPC featureMaker from /gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/cpc_small/checkpoint_25.pt
Loading checkpoint /gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/cpc_small/checkpoint_25.pt
Updating the configuration file with 
{
    "nLevelsGRU": 2
}
Loading the state dict at /gpfsssd/scratch/rech/ank

N/A% (0 of 7042) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

CPC FeatureMaker loaded!

Building CPC features and saving outputs to /gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/features/cpc_small/CV_en...


100% (7042 of 7042) |####################| Elapsed Time: 0:16:57 Time:  0:16:57


...done 7042 files in 1017.6764810085297 seconds.


In [11]:
os.makedirs("/gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/cpc_small/features/CV_en", exist_ok=True) 

%run /gpfsdswork/projects/rech/ank/ucv88ce/repos/zerospeech2021_baseline/scripts/build_CPC_features.py \
    "/gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/cpc_small/checkpoint_25.pt" \
    "/gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/fr" \
    "/gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/features/cpc_small/CV_fr" \
    --file_extension wav --gru_level 2

Building CPC features from /gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/fr

Looking for all wav files in /gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/fr
Loaded from cache /gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/fr/_seqs_cache.txt successfully
Done! Found 7795 files!

Creating the output directory at /gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/features/cpc_small/CV_fr
Writing args to /gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/features/cpc_small/CV_fr/_info_args.json

Loading CPC featureMaker from /gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/cpc_small/checkpoint_25.pt
Loading checkpoint /gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/cpc_small/checkpoint_25.pt
Updating the configuration file with 
{
    "nLevelsGRU": 2
}
Loading the state dict at /gpfsssd/scratch/rech/ank

  0% (1 of 7795) |                       | Elapsed Time: 0:00:00 ETA:   0:13:13

CPC FeatureMaker loaded!

Building CPC features and saving outputs to /gpfsssd/scratch/rech/ank/ucv88ce/projects/MultilingualCPC/checkpoints/inftrain/EN+FR/3200h/00/features/cpc_small/CV_fr...


100% (7795 of 7795) |####################| Elapsed Time: 0:16:59 Time:  0:16:59


...done 7795 files in 1019.2062411308289 seconds.
